In [1]:
from utils.util import mkdir,seed_all
from omegaconf import OmegaConf
from cprint import *
from datasets.shape_net import ShapeNet
import torch
from models.Transform2D import Transform2D
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
from utils.visualizations import save_voxels
from PIL import Image
import os
import numpy as np
import datetime
%load_ext autoreload
%load_ext tensorboard
%autoreload 2


In [2]:
import time
seed_all(111)
today = time.strftime("%Y-%m-%d")
config = OmegaConf.load("./configs/global_configs.yaml")
cprint.ok(config)
description = "all dataset gg" # Describe Experiment params here
logs_dir = config["logs_dir"]
mkdir(logs_dir)
experiment_dir = f"{logs_dir}/testtt"
mkdir(experiment_dir)
loss_log_title = "Loss Log" + today 

with open(f"{experiment_dir}/description.txt", "w") as file1:
    file1.write(description)
    
with open(f"{experiment_dir}/configs.txt", "w") as file1:
    file1.write(str(config))

with open(f"{experiment_dir}/loss_log.txt", "w") as file1:
    file1.write(loss_log_title)
    file1.write("\n")


mkdir(f"{experiment_dir}/checkpoints")
mkdir(f"{experiment_dir}/tb")

{'logs_dir': 'logs', 'is_train': True, 'name': 'with_img_transforms_plz_work', 'device': 'cuda:0', 'batch_size': 16, 'n_epochs': 100, 'print_every': 150, 'validate_every': 3000, 'save_every': 3000, 'save_every_nepochs': 10, 'model': {'lr': 0.0001, 'criterion': 'BCE', 'pos_weight': 1.3, 'encoder': {'patch_size': 16, 'sequence_length': 196, 'embedding_dim': 768, 'patch_padding': 3}, 'transformer_encoder': {'d_model': 768, 'nhead': 12, 'num_layers': 12}, 'transformer_decoder': {'d_model': 768, 'nhead': 12, 'num_layers': 8, 'num_pos_embeddings': 64}}}
- logs directory found
- logs/testtt directory found
- logs/testtt/checkpoints directory found
- logs/testtt/tb directory found


In [3]:
dataset = ShapeNet(cat="chair",is_overfit=False, nimgs=3) #Change overfit param here & cat here
print('length: ', len(dataset))
config = OmegaConf.load("./configs/global_configs.yaml")
dataset[0]
train_ds, valid_ds, test_ds = torch.utils.data.random_split(
    dataset, [6000, 778, 0])

train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        #num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

validation_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        #num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )



length:  6778


In [ ]:
from transformers import AutoImageProcessor, DeiTModel

In [4]:
model = DeiTModel.from_pretrained("facebook/deit-base-distilled-patch16-224")

NameError: name 'DeiTModel' is not defined

In [ ]:
model.device

In [ ]:
model.parameters

In [5]:
model = Transform2D()

Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTModel: ['cls_classifier.weight', 'cls_classifier.bias', 'distillation_classifier.bias', 'distillation_classifier.weight']
- This IS expected if you are initializing DeiTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTModel were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['deit.pooler.dense.bias', 'deit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

In [6]:
from utils.visualizations import visualize_png
train_loss_running = 0.
best_loss_val = np.inf
model.train()

tb_dir = f"{experiment_dir}/tb"
writer = SummaryWriter(log_dir=tb_dir)
model_checkpoint_path = f"{experiment_dir}/checkpoints"
loss_log_name = f"{experiment_dir}/loss_log.txt"
visuals_path =  f"{experiment_dir}/visuals"
last_loss = 0.
last_iou = 0.


def train_one_epoch(epoch, writer):
     global best_loss_val
     global last_loss   
     global last_iou
     train_loss_running = 0.
     train_iou_running = 0.
     iteration_count = 0
     for batch_idx, batch in tqdm(enumerate(train_dataloader)):
         ShapeNet.move_batch_to_device_float(batch, "ss")
         model.step(batch)
         metrics = model.get_metrics()
         loss = metrics["loss"]
         iou = metrics["iou"]
         train_loss_running += loss
         train_iou_running += iou
         iteration = epoch * len(train_dataloader) + batch_idx   
         iteration_count += 1
         if iteration % config['print_every'] == (config['print_every'] - 1):
            avg_train_loss = train_loss_running / iteration_count
            avg_iou = train_iou_running / iteration_count
            message = '(epoch: %d, iters: %d, loss: %.6f, iou: %.6f)' % (epoch, iteration, loss.item(), iou.item())
            with open(loss_log_name, "a") as log_file:
                log_file.write('%s\n' % message)
            print(loss)
            reconstructions = model.x
            target = batch["voxels"].squeeze(1)
            fig = save_voxels(reconstructions, target, visuals_path, iteration, is_train=True )
            writer.add_figure("Train/Reconstructions", fig, global_step=iteration, close=True, walltime=None)
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {avg_train_loss:.6f}')
            writer.add_scalar("Train/Loss", avg_train_loss, iteration)
            writer.add_scalar("Train/iou", avg_iou, iteration)
            last_loss = avg_train_loss
            last_iou = avg_iou
            train_loss_running = 0.
            train_iou_running = 0.
            iteration_count = 0
         
         if iteration % config['save_every'] == (config['save_every'] - 1):
            model.save(model_checkpoint_path, "latest")
            
        
         if iteration % config['validate_every'] == (config['validate_every'] - 1):
            cprint.ok("Running Validation")
            model.eval()
            loss_val = 0.
            iou_val = 0.
            index_batch = 0
            for batch_val in validation_dataloader:
                ShapeNet.move_batch_to_device_float(batch_val, device)
                with torch.no_grad():
                    model.inference(batch_val)
                    metrics = model.get_metrics()
                    loss_val +=  metrics["loss"]
                    iou_val += metrics["iou"]
                    index_batch += 1
            avg_loss_val = loss_val / (index_batch)
            avg_iou_val = iou_val / (index_batch)
            reconstructions = model.x
            target = batch_val["voxels"].squeeze(1)
            fig = save_voxels(reconstructions, target, visuals_path, iteration, is_train=False )
            writer.add_figure("Validation/Reconstructions", fig, global_step=iteration, close=True, walltime=None)
            
            if avg_loss_val < best_loss_val:
                model.save(model_checkpoint_path, "best")
                best_loss_val = avg_loss_val
            
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {avg_loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
            writer.add_scalar("Validation/Loss", avg_loss_val, iteration)
            #import pdb;pdb.set_trace();
            writer.add_scalars('Validation/LossComparison',
                    { 'Training' : last_loss, 'Validation' : avg_loss_val },
                     iteration)
            
            writer.add_scalars("Validation/iouComparison",
                                 { 'Training' : last_iou, 'Validation' : avg_iou_val},
                                     iteration)
            
            writer.flush()
      
            return last_loss

     




for epoch in tqdm(range(config['n_epochs'])):
    avg_loss = train_one_epoch(epoch, writer) 
    if(epoch % config["save_every_nepochs"]==0):
        model.save(model_checkpoint_path, epoch)
    model.update_lr()
    writer.close()
            
#cprint.ok(f"Visualizations saved to: {experiment_dir}/visuals")



  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

> /mnt/hdd/streakfull/3DML/Project/src/models/Transform2D.py(95)forward()
     93         #import pdb;pdb.set_trace()
     94         import pdb;pdb.set_trace()
---> 95         x = self.transformer_encoder(x)
     96         x = self.transformer_decoder(x)
     97 

ipdb> self.images.shape
torch.Size([48, 3, 224, 224])
ipdb> k = self.deit_model(self.images)
ipdb> k.shape
*** AttributeError: 'BaseModelOutputWithPooling' object has no attribute 'shape'
ipdb> last_hidden_states = k.last_hidden_state
ipdb> last_hidden_states.shape
torch.Size([48, 198, 768])
ipdb> last_hidden_states
tensor([[[ 2.0033e-01,  1.3465e-01,  7.0917e-02,  ..., -5.1942e-02,
          -3.3614e-01, -8.1730e-01],
         [ 8.3741e-01,  5.3440e-01, -1.2683e+00,  ..., -3.1051e-01,
          -1.1625e+00, -3.5187e-01],
         [-2.5050e-02,  5.7442e-02, -8.7433e-01,  ..., -3.1038e-01,
          -4.5043e-01, -4.1194e-03],
         ...,
         [ 4.6494e-01,  5.7515e-01, -6.7269e-01,  ..., -7.2884e-01,
          -3.8007e